In [1]:
import pandas as pd
import string
import re
import nltk
import pymorphy2

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
nltk.download('word_tokenize')
nltk.download('punkt')
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Error loading word_tokenize: Package 'word_tokenize' not
[nltk_data]     found in index
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nudan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
pip install pymorphy2

Note: you may need to restart the kernel to use updated packages.


In [3]:
df =pd.read_csv(r"C:\Users\nudan\Спец курс\Pdf parsing\dataset2.csv")

In [4]:
df.head()

,Unnamed: 0,namecompany,description,rating,activity,date_publish,text_article
0,0,Timeweb Cloud,Облачная платформа для разработчиков и бизнеса,1572.77,"Веб-разработка, Домены и хостинг, Веб-сервисы",2023-03-27,В своём блоге я рассмотрел устройство и назнач...
1,1,OTUS,Цифровые навыки от ведущих экспертов,795.43,"Консалтинг и поддержка, Рекрутинг и HR, Произв...",2023-03-27,Автор статьи: Роман КозловРуководитель курса B...
2,2,Онлайн Патент,Ваш личный патентный офис,220.74,"Консалтинг и поддержка, Веб-сервисы",2023-03-27,Рынок биологически активных добавок(БАД) с каж...
3,3,OTUS,Цифровые навыки от ведущих экспертов,795.43,"Консалтинг и поддержка, Рекрутинг и HR, Произв...",2023-03-27,Автор статьи: Александр КолесниковВирусный ана...
4,4,VK,"Технологии, которые объединяют",568.63,"Мобильные технологии, Веб-сервисы, Игры и разв...",2023-03-27,"Всем привет! Меня зовут Артём Пузанков, я DevS..."


In [5]:
morph = pymorphy2.MorphAnalyzer()

In [6]:
def preprocess_text(data, stopwords=stopwords.words("russian")):
    stopwords.extend(['либо', 'это', 'мб', 'далее', 'дв', 'свой', 'ваш','всё', 'очень', 'её', 'ещё', 'вообще', 'наш', 'который',])
    text = re.sub('ё','е', data.lower())
    text = re.sub(r"([.,!?])", r" \1 ",text)
    text = re.sub(r"[^а-яА-Я\s]+", "",text)
    text = text.strip()
    text = [w for w in text.split() if w not in stopwords] 
    text = [w for w in text if len(w) >=3]
    return ' '.join(text)

In [7]:
def lemmatization_text(data, morph=morph):
    result = ' '.join([morph.parse(x)[0].normal_form for x in data.split()])
    return result

In [8]:
def get_result(data, morph=morph, stopwords=stopwords.words("russian")):
    result = preprocess_text(data=data)
    result = lemmatization_text(result)
    return result

In [9]:
stopword = nltk.corpus.stopwords.words('russian')

In [10]:
stopword.extend(['либо', 'это', 'мб', 'далее', 'дв', 'свой', 'ваш','всё', 'очень', 'её', 'ещё', 'вообще', 'наш', 'который',])    

In [11]:
def transform_data(data: pd.Series) -> list:
    result = [get_result(data=i) for i in data]
    return result

### Векторизация текстовых данных

#### 1. Прямое кодирование (one-hot encoding) 

__Прямое кодирование__ (one-hot encoding) - самый простой способ преобразования токенов в тензоры (координатное представление вектора) и выполняется следующим способом:

- каждый токен представляет бинарный вектор (значения 0 или 1)
- единица ставится тому элементу, который соответствует номеру токена в словаре

Проблемой прямого кодирования является размерность

#### 2. Векторизация. Мешок слов

__Модель мешка слов__ (Bag of Words, BoW) - для документа формируется вектор размерности словаря и записывается признак насколько часто слово встречается в нем. Этот метод не учитывает важность того или иного токена.

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
text = df['text_article']

In [14]:
%%time
df_res = transform_data(text)

CPU times: total: 1min 26s
Wall time: 1min 26s


In [15]:
len(df_res)

438

In [16]:
vectorizer = CountVectorizer(analyzer='word', stop_words=stopword, ngram_range=(1, 3), min_df=2)
count_matrix = vectorizer.fit_transform(df_res)

In [17]:
count_matrix.toarray()[:2,:200]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 

In [18]:
count_matrix.shape

(438, 53381)

In [19]:
vectorizer.get_feature_names_out()[:50]

array(['аба', 'аббревиатура', 'абзац', 'абитуриент', 'абонемент',
       'абонент', 'абсолютно', 'абсолютно весь', 'абсолютно любой',
       'абсолютно никакой', 'абсолютно новый', 'абсолютно одинаковый',
       'абсолютно разный', 'абсолютный', 'абсолютный большинство',
       'абсолютный разница', 'абстрагировать', 'абстрагироваться',
       'абстрактный', 'абстрактный понятие', 'абстрактный рассуждение',
       'абстрактный синтаксический', 'абстрактный синтаксический дерево',
       'абстракция', 'абстракция работа', 'абстракция работа устройство',
       'абстракция специальный', 'абстракция специальный программа',
       'абстракция язык', 'аварийный', 'авария', 'аватар', 'август',
       'август год', 'авиабилет', 'авиакомпания', 'авиация', 'авить',
       'авось', 'аврал', 'австралийский', 'австралия', 'авто', 'автобус',
       'автозагрузка', 'автомасштабирование', 'автомат', 'автоматизатор',
       'автоматизация', 'автоматизация браузер'], dtype=object)

In [20]:
len(vectorizer.get_feature_names_out())

53381

#### 3. Векторизация. TF-IDF

TF-IDF (TF - Term Frequency - как часто слово встречается в документе (важность слова в контексте документа), IDF - Inverce Document Frequency) - статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса. Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции (важность слова возрастает, если он используется в одном документе и не используется в других).

TF считается для токенов документа, IDF - токенов всего корпуса. В TF-IDF редкие слова и слова, которые встречаются во всех документах, несут мало информации

Частота слова (Term Frequency):
Term Frequency высчитывает __вероятность найти какое-то слово в документе.__ Ну, например, мы хотим узнать, какова вероятрность найти слово wi в документе dj.

Term Frequency (wi, dj) = Количество раз, которое wi встречается в dj / Общее число слов в dj

Обратная частота документа (Inverse Document Frequency):В логике IDF, если слово встречается во всех документах, оно не очень полезно. Так определяется, __насколько уникально слово во всем корпусе.__

IDF(wi, Dc) = log(N/ni)

Здесь Dc = Все документы в корпусе,

N = Общее число документов,

ni = документы, которые содержат слово (wi).

Если wi встречается в корпусе часто, значение IDF снижается.

Если wi используется не часто, то ni снижается и вследствие этого значение IDF возрастает.

TF(wi, dj) * IDF(wi, Dc)

TF-IDF — умножение значений TF и IDF. __Больший вес получат слова, которые встречаются в документе чаще, чем во всем остальном корпусе.__

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.01, stop_words=stopwords.words("russian"),
                                 ngram_range=(1,3))

In [23]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, 
                                 min_df=0.01, stop_words=stopwords.words("russian"),
                                 ngram_range=(1,3))

In [24]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(df_res)

CPU times: total: 1.5 s
Wall time: 1.51 s


In [25]:
type(tfidf_matrix)

scipy.sparse._csr.csr_matrix

In [26]:
tfidf_matrix.shape

(438, 10680)

In [27]:
from sklearn.decomposition import LatentDirichletAllocation

In [28]:
RAND = 10
lda_model = LatentDirichletAllocation(learning_method='online', random_state=RAND, n_jobs=-1, n_components=11)

In [29]:
lda_model.fit(count_matrix)

LatentDirichletAllocation(learning_method='online', n_components=11, n_jobs=-1,
                          random_state=10)

In [30]:
len(lda_model.components_)

11

In [31]:
lda_model.components_

array([[ 3.05734001,  7.13332506, 17.52755145, ..., 46.99804742,
         0.95278094,  1.72869764],
       [ 0.10508665,  0.1173822 ,  0.11131075, ...,  0.14346434,
         0.10677013,  0.1024581 ],
       [ 0.10295781,  0.10447591,  0.10683376, ...,  0.10832202,
         0.10543611,  0.10755306],
       ...,
       [ 0.1031781 ,  0.1082408 ,  0.10521158, ...,  0.1082272 ,
         0.10531563,  0.10225368],
       [ 0.1036638 ,  0.10578227,  0.10657105, ...,  0.10264297,
         0.10249063,  0.10391336],
       [ 0.10362396,  0.11053884,  0.10318543, ...,  0.10579743,
         0.10289895,  0.10403483]])

In [32]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [33]:
import pyLDAvis.sklearn

In [34]:
from __future__ import division

import pandas as pd
import pyLDAvis


In [35]:
pip install graphlab

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement graphlab (from versions: none)
ERROR: No matching distribution found for graphlab


In [37]:
pip install --upgrade graphlab-create

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement graphlab-create (from versions: none)
ERROR: No matching distribution found for graphlab-create


In [38]:
pyLDAvis.enable_notebook()

In [39]:
panel = pyLDAvis.sklearn.prepare(lda_model, count_matrix, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      -90.473381  -43.938328       1        1  91.198456
1      124.217400   44.315674       2        1   3.775743
3     -100.679260   41.628422       3        1   2.611516
4       54.085712    0.495221       4        1   1.454423
10      49.066151   89.579781       5        1   0.427004
2      118.811287  -50.354965       6        1   0.418634
8      -35.317219  110.101448       7        1   0.052791
9       -6.179249  -46.312103       8        1   0.039269
5       52.805256 -105.200668       9        1   0.007388
7      -39.052906 -120.649307      10        1   0.007388
6      -19.077660   29.106888      11        1   0.007388, topic_info=             Term         Freq        Total Category  logprob  loglift
6083          год  1302.000000  1302.000000  Default  30.0000  30.0000
47205        тест   701.000000   701.000000  Default  29.0000  29.0000
41662     система  1824.000000  1824.000000  Default  28.0000  28.0000
16047     команда  1592.000000  1592.000000  Default  27.0000  27.0000
33580  приложение  1361.000000  1361.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
41662     система     0.001878  1824.164909  Topic11  -9.9609  -4.2736
40829     сделать     0.001745   946.525448  Topic11 -10.0339  -3.6905
7182         дать     0.001753  1166.433890  Topic11 -10.0294  -3.8949
42785      случай     0.001734   995.252938  Topic11 -10.0403  -3.7471
23168       новый     0.001736  1395.565294  Topic11 -10.0396  -4.0844

[805 rows x 6 columns], token_table=       Topic      Freq                   Term
term                                         
225        6  1.071929  администратор кластер
303        1  0.992909                аккаунт
323        1  0.219326                  актив
323        4  0.767639                  актив
499        1  0.630982           американский
...      ...       ...                    ...
53345      5  0.069629        языковой модель
53359      1  0.516389                 япония
53359      4  0.473356                 япония
53360      1  0.188815               японский
53360      4  0.809209               японский

[1040 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 5, 11, 3, 9, 10, 6, 8, 7])

In [40]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, count_matrix, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      -90.473381  -43.938328       1        1  91.198456
1      124.217400   44.315674       2        1   3.775743
3     -100.679260   41.628422       3        1   2.611516
4       54.085712    0.495221       4        1   1.454423
10      49.066151   89.579781       5        1   0.427004
2      118.811287  -50.354965       6        1   0.418634
8      -35.317219  110.101448       7        1   0.052791
9       -6.179249  -46.312103       8        1   0.039269
5       52.805256 -105.200668       9        1   0.007388
7      -39.052906 -120.649307      10        1   0.007388
6      -19.077660   29.106888      11        1   0.007388, topic_info=             Term         Freq        Total Category  logprob  loglift
6083          год  1302.000000  1302.000000  Default  30.0000  30.0000
47205        тест   701.000000   701.000000  Default  29.0000  29.0000
41662     система  1824.000000  1824.000000  Default  28.0000  28.0000
16047     команда  1592.000000  1592.000000  Default  27.0000  27.0000
33580  приложение  1361.000000  1361.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
41662     система     0.001878  1824.164909  Topic11  -9.9609  -4.2736
40829     сделать     0.001745   946.525448  Topic11 -10.0339  -3.6905
7182         дать     0.001753  1166.433890  Topic11 -10.0294  -3.8949
42785      случай     0.001734   995.252938  Topic11 -10.0403  -3.7471
23168       новый     0.001736  1395.565294  Topic11 -10.0396  -4.0844

[805 rows x 6 columns], token_table=       Topic      Freq                   Term
term                                         
225        6  1.071929  администратор кластер
303        1  0.992909                аккаунт
323        1  0.219326                  актив
323        4  0.767639                  актив
499        1  0.630982           американский
...      ...       ...                    ...
53345      5  0.069629        языковой модель
53359      1  0.516389                 япония
53359      4  0.473356                 япония
53360      1  0.188815               японский
53360      4  0.809209               японский

[1040 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 5, 11, 3, 9, 10, 6, 8, 7])

In [41]:
import graphlab as gl
import pyLDAvis.graphlab

ModuleNotFoundError: No module named 'graphlab'

In [42]:
import pyLDAvis
import pyLDAvis.gensim

data = pyLDAvis.sklearn.prepare(lda_model, count_matrix, vectorizer)
# Разрешить отображение визуализации в блокноте
pyLDAvis.display(data)

In [43]:
tf_vectorizer = CountVectorizer(max_df=0.8, max_features=7474 ,
                                 min_df=0.01, stop_words=stopword,
                                 ngram_range=(1,3))
dtm_tf = tf_vectorizer.fit_transform(df_res)
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(df_res)

In [46]:
import pyLDAvis
import pyLDAvis.gensim

data = pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, mds='mmds')
# Разрешить отображение визуализации в блокноте
pyLDAvis.display(data)

In [45]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=11, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=11, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=11, random_state=0)

In [47]:
for i,topic in enumerate(lda_model.components_):
    print(f'Top 10 words for topic #{i + 1}:')
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #1:
['новый', 'работать', 'команда', 'система', 'время', 'код', 'данные', 'работа', 'мочь', 'использовать']


Top 10 words for topic #2:
['человек', 'поверхность', 'россия', 'информация', 'информационный', 'учёный', 'лицо', 'робот', 'система', 'игра']


Top 10 words for topic #3:
['метка', 'юнит', 'мобильный', 'продуктовый команда', 'подписка', 'логистика', 'продуктовый', 'приложение', 'продукт', 'чемпион']


Top 10 words for topic #4:
['эмулятор', 'компьютер', 'диск', 'чип', 'использовать', 'команда', 'система', 'файл', 'образ', 'устройство']


Top 10 words for topic #5:
['лечение', 'лекарство', 'заявка', 'патентный', 'компания', 'препарат', 'японский', 'доллар', 'патент', 'год']


Top 10 words for topic #6:
['модель', 'получить', 'время', 'также', 'год', 'мочь', 'приложение', 'использовать', 'трейс', 'работа']


Top 10 words for topic #7:
['задача', 'например', 'использовать', 'год', 'файл', 'данные', 'время', 'мочь', 'приложение', 'работа']


Top 10 words for 

In [48]:
for i,topic in enumerate(lda_tfidf.components_):
    print(f'Top 10 words for topic #{i + 1}:')
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #1:
['год мир', 'давать ряд', 'вёрстка', 'вроде сделать', 'база данные файл', 'версия работа', 'аниме', 'воспитать', 'выше вероятность', 'аудит мониторинг']


Top 10 words for topic #2:
['вредоносный программный', 'близкий десять', 'безопасность архитектор инфраструктурный', 'аудит мониторинг', 'данный случай строка', 'выстреливать', 'движок контейнеризация', 'безопасность возникать', 'движок версия', 'вышесказанный']


Top 10 words for topic #3:
['браузер рисунок', 'выглядеть следующий образ', 'группа внутри', 'видеть впервые', 'воздействие получить', 'база осуществляться', 'выходной дать', 'активный год', 'выглядеть совершенно', 'выглядеть словно']


Top 10 words for topic #4:
['восприниматься', 'вид модуль', 'асимметричный', 'благодарный', 'автоматически переключиться', 'ами', 'давать сравнимый', 'вид образ', 'великое множество', 'давать результат']


Top 10 words for topic #5:
['вопрос возникать', 'выводиться сообщение', 'авторизовать доступ', 'браузер движок

In [49]:
for i,topic in enumerate(lda_tf.components_):
    print(f'Top 10 words for topic #{i + 1}:')
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #1:
['веха', 'вебинар эксперт', 'высокий яркость', 'герасимович', 'внешне', 'вариант решение задача', 'браузер движок', 'безумно интересно', 'данные самый', 'данные поток']


Top 10 words for topic #2:
['вопрос нужно', 'вне зависимость', 'быстрый старт', 'бывать кандидат', 'булочка', 'важно уделять', 'гаджет', 'баланс', 'выходной параметр нужно', 'герасимович']


Top 10 words for topic #3:
['генерирование', 'больший творчество процесс', 'двадцать', 'время прийти', 'агрессивный оптимизация', 'выходной дать', 'большинство потребность', 'ваня', 'авторизация помощь', 'данные решать задача']


Top 10 words for topic #4:
['возраст', 'вместе позволять углубить', 'глубоко изучать', 'выжать максимум', 'гайда устать', 'видеокурс мег подписка', 'год позиция', 'данные самый', 'вебверсия', 'гдеть рядом']


Top 10 words for topic #5:
['градиент', 'алгоритм алгоритм', 'вместе позволять углубить', 'данные самый', 'выбор драйвер', 'весь мир', 'административный', 'данный модель', 

In [50]:
import numpy as np
# Определяем топ ключевых слов в каждой теме 
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=100):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=tfidf_vectorizer, lda_model=lda_tfidf, n_words=100)        

# Создаем датасет тема - ключевое слово 
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

In [51]:
df_topic_keywords.head()

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,...,Word 90,Word 91,Word 92,Word 93,Word 94,Word 95,Word 96,Word 97,Word 98,Word 99
Topic 0,вычислительный техника,рубль первый,писать код,воспринимать,логично,год создавать,посетить,связаться,товарный,создавать уникальный,...,переходный,макрос,коммутация,ведомство,фишинговый,кража,создать первый,сливаться,поэтому давать,санкция
Topic 1,санкция,эффективно делиться,делиться результат работа,эффективно делиться результат,результат работа хвастовство,хвастовство,вычислительный техника,делиться результат,закрыться,подобный образ,...,расширяться,партнёр,облачный провайдер,конфигуратор,несколько разный,слабый,постройка,война,начало год,изготовление
Topic 2,продление,продлить,вдоль,рис,голубой,оранжевый,натуральный,также отметить,продлевать,иной слово,...,плоскость,строиться,исходный,вычислить,описать выше,рассуждение,сводиться,отнести,нововведение,граница
Topic 3,ткань,краска,нанесение,толстый,вкратце,альтернативный вариант,зазор,выдерживать,наложить,пластика,...,какоголибо,каждый тип,препарат,стол,примитивный,рамка статья,рекомендоваться использовать,органический,головка,печать
Topic 4,контейнер,контроллер,запускать,сертификат,редактор,баг,индекс,атрибут,программист,паттерн,...,вредонос,пропустить,добавляться,поменять,встретить,пустой,архив,сохранение,сеньор,писать код


In [52]:
prize = ['Государство и Общество','СМИ и массовые коммуникации','Образование и Кадры', 'Экономика и Бизнес', 'Путешествия по РФ', 'Наука, технологии и инновации', 'Здоровье и Отдых', 'Культурные проекты в сети', 'Народов много - Родина одна', 'Туризм и индустрия гостеприимства', 'Подкасты и цифровой контент']

In [53]:
num_clusters = 11

# Метод к-средних - KMeans
from sklearn.cluster import KMeans
km = KMeans(n_clusters=num_clusters)

In [54]:
%%time
cm = km.fit(count_matrix)

CPU times: total: 7 s
Wall time: 1.64 s


In [58]:
clusters1 = cm.labels_.tolist()
clusters1[:10]

[2, 1, 2, 2, 2, 2, 2, 2, 2, 2]